In [5]:
!pip install -q transformers

In [6]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix , classification_report
import pandas as pd
import seaborn as sns

import tensorflow as tf
import json

import numpy as np
import pandas as pd


import os
import re

In [7]:
# jsonDataString = "{"
# DataTypes = []
# EmotionData= []
# dataDir = ["data/"]
# for dataList in dataDir:
#   for dirNames in os.listdir(os.path.join(dataList)):
#     jsonDataString+=dirNames+':[{'
#     temp2 = ""
#     for emoDir in os.listdir(os.path.join(dataList,dirNames)):
#       jsonDataString += emoDir+':['
#       for filesName in os.listdir(os.path.join(dataList,dirNames,emoDir)):
#         with open(os.path.join(dataList,dirNames,emoDir, filesName) ,'r') as f:
#           jsonDataString+=re.sub(r"\n", "", f.read())+','
          
#       jsonDataString = jsonDataString[0:len(jsonDataString)-1]+'],'
#     jsonDataString =jsonDataString[0:len(jsonDataString)-1]+'}],'
  

#       # for filesName in os.listdir(os.path.join(dataList,dirNames,emoDir)):
#       #   print('',end='')
#     #with open(os.path.join(os.getcwd(), filename), 'r') as f:
# jsonDataString = jsonDataString[0:len(jsonDataString)-1]+'}' 
# print(jsonDataString)

In [8]:
TestTrainData = json.load(open('/home/data.json'))
TrainData = TestTrainData['train'][0]
TestData = TestTrainData['test'][0]
FormattedTrainData=[]
FormattedTrainLabel=[]
FormattedTestData=[]
FormattedTestLabel=[]

for i in ((TestData)):
  # if((list((TestData.keys())).index(i))<2):
  for ii in TestData[i]:
      FormattedTestData.append([(list((TestData.keys())).index(i)),ii])
      FormattedTestLabel.append(((list((TrainData.keys())).index(i))))

for i in ((TrainData)):
  # if((list((TrainData.keys())).index(i))<2):
  for ii in TrainData[i]:
        FormattedTrainData.append([(list(TrainData.keys()).index(i)),ii])
        FormattedTrainLabel.append((list(TrainData.keys()).index(i)))

FormattedTrainData = pd.DataFrame(np.array((FormattedTrainData)), columns = ['label','message']).sample(frac = 1)
FormattedTestData =  pd.DataFrame(np.array((FormattedTestData)), columns = ['label','message']).sample(frac = 1)
FormattedTrainData.head()
FormattedTestData.head()

,label,message
132,0,i just feel like someone out there has to list...
842,3,i feel somewhat fake in the group
922,3,ive been feeling a bit messy but im hoping thi...
380,1,i am feeling the past few days a little distre...
1079,3,i just feel totally useless today


In [9]:
print(FormattedTrainData['label'])


2451    1
6624    3
5277    2
6052    3
492     0
       ..
581     0
8544    3
3544    2
6498    3
2628    1
Name: label, Length: 8687, dtype: object


In [10]:
training_sentences = list(FormattedTrainData['message'])
validation_sentences = list(FormattedTestData['message'])
training_labels = list(pd.to_numeric(FormattedTrainData['label'], downcast='integer'))
validation_labels =  list(pd.to_numeric(FormattedTestData['label'], downcast='integer'))

print(type(training_labels[0]))

<class 'int'>


In [11]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))


In [13]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=4)

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])
hist = model.fit(train_dataset.batch(128),
          epochs=1,
          batch_size=128,
          validation_data=val_dataset.shuffle(100).batch(128))

52/68 [=====================>........] - ETA: 15:31 - loss: 0.8724 - accuracy: 0.6411

In [ ]:
print(hist.history.keys())
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')true_output
plt.legend(['train', 'test'], loc='upper left')
plt.show()


plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.summary()
tf.keras.utils.plot_model(model)

model.save_pretrained('/home/emotion_Model')

In [ ]:
!mkdir /home/emotion_Model

In [ ]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("/home/emotion_Model")

In [ ]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("/home/emotion_Model")

predict_input =  DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased').encode(i,
                                truncation=True,
                                padding=True,
                                return_tensors="tf")
tf_output = loaded_model.predict(predict_input)[0]
tf_prediction = list(tf.nn.softmax(tf_output)[0].numpy())
predition = tf_prediction.index((list(tf_prediction)))

# predict_output.append(list(TestData)[predition])
# print(predict_output)

In [ ]:

predict_output=[]
for i in validation_sentences:
  predict_input = tokenizer.encode(i,
                                  truncation=True,
                                  padding=True,
                                  return_tensors="tf")
  tf_output = loaded_model.predict(predict_input)[0]
  tf_prediction = list(tf.nn.softmax(tf_output)[0].numpy())
  predition = tf_prediction.index(max(list(tf_prediction)))
  predict_output.append(list(TestData)[predition])
print(predict_output)

In [ ]:
true_output = []
for i in list(pd.to_numeric(validation_labels)):
  true_output.append(list(TestData)[i])
print(true_output)

# print(tf_prediction)
# print(TestData.keys())
# print(TrainData.keys())

In [ ]:
from sklearn.metrics import f1_score
# f1_score(true_output, pimport matplotlib.pyplot as pltredict_output, average='macro')
print(classification_report(true_output, predict_output))

In [ ]:


# Source code credit for this function: https://gist.github.com/shaypal5/94c53d765083101efc0240d776a23823
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.
    
    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. 
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.
        
    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('Truth')
    plt.xlabel('Prediction')
       225
        Love       0.99      0.98      0.99       160


In [ ]:
val = confusion_matrix(true_output,predict_output)  
print_confusion_matrix(val,TestData.keys())